<a href="https://colab.research.google.com/github/shikharkyadesultory/Eff-mass-Frohlic-omegaLO/blob/main/CMS(ASE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install ase

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db import connect
from ase.eos import calculate_eos

db = connect('bulk.db')
for symb in ['Al', 'Ni', 'Cu', 'Pd', 'Ag', 'Pt', 'Au']:
    atoms = bulk(symb, 'fcc')
    atoms.calc = EMT()
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()

    atoms.cell *= (v / atoms.get_volume())**(1 / 3)
    atoms.get_potential_energy()
    db.write(atoms, bm=B)

    print(f"Details for {symb}:")
    print(f"  Volume: {v}")
    print(f"  Energy: {e}")
    print(f"  Bulk Modulus: {B}")

Details for Al:
  Volume: 15.931923303048151
  Energy: -0.004882564299247605
  Bulk Modulus: 0.24913771368349813
Details for Ni:
  Volume: 10.600868900812102
  Energy: -0.013311738418053665
  Bulk Modulus: 1.1046241310550273
Details for Cu:
  Volume: 11.565374510212838
  Energy: -0.00703647873130997
  Bulk Modulus: 0.8392875566772626
Details for Pd:
  Volume: 14.588455633485026
  Energy: -0.0002691205244644834
  Bulk Modulus: 1.1183158315343873
Details for Ag:
  Volume: 16.774808232027375
  Energy: -0.00036686258794560445
  Bulk Modulus: 0.6251315997176192
Details for Pt:
  Volume: 15.079861135270544
  Energy: -0.00014870583541437554
  Bulk Modulus: 1.736401895011947
Details for Au:
  Volume: 16.6835259635432
  Energy: -0.0001350522580452207
  Bulk Modulus: 1.0850176028639944


In [53]:
from ase.build import add_adsorbate,fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.db import connect
from ase.optimize import BFGS

db1 = connect('bulk.db')
db2 = connect('ads.db')

def run(symb , a , n ,ads):
  atoms = fcc111(symb,(1,1,n),a=a)
  add_adsorbate(atoms , ads , height = 1.0 , position ='fcc')

  #constraints on all atoms except the adsorbate
  fixed = list(range(len(atoms) - 1))
  atoms.constraints = [FixAtoms(indices=fixed)]

  atoms.calc = EMT()
  opt = BFGS(atoms, logfile = None)
  opt.run(fmax = 0.01)
  potential_energy = atoms.get_potential_energy()
  print(f"Potential energy: {potential_energy}")

  # Print atomic positions
  print("Atomic positions:")
  for atom in atoms:
    print(f"{atom.symbol} {atom.position}")

  return atoms

for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        for ads in 'CNO':
            atoms = run(symb, a, n, ads)
            db2.write(atoms, layers=n, surf=symb, ads=ads)

Potential energy: 0.6779345525889684
Atomic positions:
Al [0. 0. 0.]
C [1.41220498 0.81533693 1.25584007]
Potential energy: 0.464036583947971
Atomic positions:
Al [0. 0. 0.]
N [1.41220498 0.81533693 1.20937094]
Potential energy: 0.7051746586906389
Atomic positions:
Al [0. 0. 0.]
O [1.41220498 0.81533693 1.54691737]
Potential energy: 0.6823336645147431
Atomic positions:
Al [-5.22620829e-17  1.63067385e+00  0.00000000e+00]
Al [0.         0.         2.30612108]
C [1.41220498 0.81533693 3.56834388]
Potential energy: 0.4770641359148504
Atomic positions:
Al [-5.22620829e-17  1.63067385e+00  0.00000000e+00]
Al [0.         0.         2.30612108]
N [1.41220498 0.81533693 3.52530454]
Potential energy: 0.7048812094815267
Atomic positions:
Al [-5.22620829e-17  1.63067385e+00  0.00000000e+00]
Al [0.         0.         2.30612108]
O [1.41220498 0.81533693 3.85882302]
Potential energy: 0.6774273652982354
Atomic positions:
Al [1.41220498 0.81533693 0.        ]
Al [-5.22620829e-17  1.63067385e+00  2.30

**Formation of ZrS2**

In [54]:
#Code to form ZrS2 and determine the basic properties of the material
from ase.build import mx2 #to create layered transition metal
from ase.calculators.emt import EMT
from ase.db import connect
formula = 'ZrS2'
structure = '1T' #Octahedral crystal structure
atoms = mx2(formula, kind=structure, a=3.66)#approximate lattice parameters
atoms.calc = EMT()
print(f"Chemical Formula : {atoms.get_chemical_formula()}")
print(f"Number of atoms: {len(atoms)}")
print(f"Lattice vectors:\n{atoms.cell}")
print(f"Atom positions:\n{atoms.get_positions()}")
db = connect('ZrS2.db')

Chemical Formula : S2Zr
Number of atoms: 3
Lattice vectors:
Cell([[3.66, 0.0, 0.0], [-1.83, 3.1696529778510456, 0.0], [0.0, 0.0, 0.0]])
Atom positions:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.83000000e+00  1.05655099e+00  1.59500000e+00]
 [-6.29126381e-18  2.11310199e+00 -1.59500000e+00]]


**Adsorbing Au on the ZrS2 surface**

In [55]:
!apt-get update
!apt-get install -y build-essential libfftw3-dev libxc-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,211 kB]
Fetched 3,596 kB in 2s (1,760 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubunt

In [56]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython']


In [57]:
!{sys.executable} -m pip uninstall gpaw

Found existing installation: gpaw 25.1.0
Uninstalling gpaw-25.1.0:
  Would remove:
    /usr/local/bin/gpaw
    /usr/local/bin/gpaw-analyse-basis
    /usr/local/bin/gpaw-basis
    /usr/local/bin/gpaw-plot-parallel-timings
    /usr/local/bin/gpaw-runscript
    /usr/local/bin/gpaw-setup
    /usr/local/bin/gpaw-upfplot
    /usr/local/lib/python3.11/dist-packages/_gpaw.cpython-311-x86_64-linux-gnu.so
    /usr/local/lib/python3.11/dist-packages/gpaw-25.1.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/gpaw/*
Proceed (Y/n)? Y
Y
  Successfully uninstalled gpaw-25.1.0


In [1]:
!pip3 install gpaw

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --no-cache-dir --prefix /usr/local gpaw

In [2]:
from ase.build import add_adsorbate
from ase.calculators.emt import EMT
from ase.db import connect
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from ase.build import mx2 # Import mx2 here

formula = 'ZrS2'
structure = '1T' #Octahedral Cyrstal Structure Assume
a_ZrS2 = 3.66 #APPROX
vacuum = 8.0 #adding up to avoid interaction with periodic instants

ZrS2_layer = mx2(formula,kind = structure ,a=a_ZrS2 , size=(2,2,1),vacuum = vacuum)
ZrS2_layer.calc = EMT()
print("ZrS2 layer :")
print(f"Chemical Formula : {ZrS2_layer.get_chemical_formula()}")
print(f"Number of atoms: {len(ZrS2_layer)}")
print(f"Lattice vectors:\n{ZrS2_layer.cell}")
print(f"Atom positions:\n{ZrS2_layer.get_positions()}")

ZrS2 layer :
Chemical Formula : S8Zr4
Number of atoms: 12
Lattice vectors:
Cell([[7.32, 0.0, 0.0], [-3.66, 6.339305955702091, 0.0], [0.0, -0.0, 19.19]])
Atom positions:
[[ 0.00000000e+00  0.00000000e+00  9.59500000e+00]
 [ 1.83000000e+00  1.05655099e+00  1.11900000e+01]
 [-6.29126381e-18  2.11310199e+00  8.00000000e+00]
 [-1.83000000e+00  3.16965298e+00  9.59500000e+00]
 [-2.22044605e-16  4.22620397e+00  1.11900000e+01]
 [-1.83000000e+00  5.28275496e+00  8.00000000e+00]
 [ 3.66000000e+00  0.00000000e+00  9.59500000e+00]
 [ 5.49000000e+00  1.05655099e+00  1.11900000e+01]
 [ 3.66000000e+00  2.11310199e+00  8.00000000e+00]
 [ 1.83000000e+00  3.16965298e+00  9.59500000e+00]
 [ 3.66000000e+00  4.22620397e+00  1.11900000e+01]
 [ 1.83000000e+00  5.28275496e+00  8.00000000e+00]]


In [3]:
adsorb = 'Au'
adsorp_ht = 2.0
adsorp_st = (0,0) #top position
adsorp_sys = ZrS2_layer.copy() #avoiding adultaration while adsorbing
add_adsorbate(adsorp_sys ,adsorb ,height = adsorp_ht, position = adsorp_st)
print(f"Chemical Formula : {adsorp_sys.get_chemical_formula()}")
print(f"Number of atoms: {len(adsorp_sys)}")
print(f"Atom types :\n{adsorp_sys.get_chemical_symbols()}")
print(f"Atom positions:\n{adsorp_sys[-1].position}")

Chemical Formula : AuS8Zr4
Number of atoms: 13
Atom types :
['Zr', 'S', 'S', 'Zr', 'S', 'S', 'Zr', 'S', 'S', 'Zr', 'S', 'S', 'Au']
Atom positions:
[ 0.    0.   13.19]


In [4]:
!which gpaw

/usr/local/bin/gpaw


In [5]:
import sys
import os
import subprocess

# Find gpaw using which command
try:
    gpaw_exec = subprocess.check_output(['which', 'gpaw']).decode().strip()
except subprocess.CalledProcessError:
    raise FileNotFoundError("Could not locate the gpaw executable. "
                            "Please make sure gpaw is installed and in your PATH.")
gpaw_data_dir = '/usr/local/lib/python3/dist-packages/gpaw/data'

# Create the directories if they don't exist:
os.makedirs(gpaw_data_dir, exist_ok=True)

!{gpaw_exec} install-data {gpaw_data_dir}

Available setups and pseudopotentials
  [*] https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.1.0.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.11271.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.9672.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.8.7929.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.6.6300.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.5.3574.tar.gz

Selected gpaw-setups-24.11.0.tar.gz.  Downloading...
Extracting tarball into /usr/local/lib/python3/dist-packages/gpaw/data
Setups installed into /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0.
Register this setup path in /root/.gpaw/rc.py? [y/n] y
It looks like the path is already registered in /root/.gpaw/rc.py.
File will not be modified at this time.
Installation compl

In [6]:
!gpaw info

 -------------------------------------------------------------------------------------------------
| python-3.11.12    /usr/bin/python3                                                              |
| gpaw-25.1.0       /usr/local/lib/python3.11/dist-packages/gpaw/                                 |
| ase-3.25.0        /usr/local/lib/python3.11/dist-packages/ase/                                  |
| numpy-2.0.2       /usr/local/lib/python3.11/dist-packages/numpy/                                |
| scipy-1.15.2      /usr/local/lib/python3.11/dist-packages/scipy/                                |
| libxc-5.1.7       yes                                                                           |
| _gpaw             /usr/local/lib/python3.11/dist-packages/_gpaw.cpython-311-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                           |
| OpenMP enabled    no                                                                            |
|

In [ ]:
!export GPAW_SETUP_PATH=/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0

In [ ]:
import os
import ase.io
os.environ['GPAW_SETUP_PATH'] = '~//gpaw-setups-24.11.0'
gpaw_data_path = os.path.join(os.path.dirname(ase.__file__), '..', 'gpaw-data')
# os.environ['GPAW_SETUP_PATH'] = gpaw_data_path

In [ ]:
ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/

Ag.11.dzp.basis.gz  In.GLLBSC.gz                        Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz                           Pt.GLLBSC.gz
Ag.11.LDA.gz        In.PBE.gz                           Pt.LDA.gz
Ag.11.PBE.gz        In.revPBE.gz                        Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz                          Pt.revPBE.gz
Ag.11.RPBE.gz       I.PBE.gz                            Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz                   Rb.dzp.basis.gz
Ag.GLLBSC.gz        Ir.9.GLLBSC.gz                      Rb.GLLBSC.gz
Ag.LDA.gz           Ir.9.LDA.gz                         Rb.LDA.gz
Ag.PBE.gz           Ir.9.PBE.gz                         Rb.PBE.gz
Ag.revPBE.gz        Ir.9.revPBE.gz                      Rb.revPBE.gz
Ag.RPBE.gz          Ir.9.RPBE.gz                        Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz                     Re.dzp.basis.gz
Al.GLLBSC.gz        I.revPBE.gz                         Re.GLLBSC.gz
Al.LDA.gz           Ir.GLLBSC.gz         

In [ ]:
!gunzip /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE.gz

In [ ]:
ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/

Ag.11.dzp.basis.gz  In.GLLBSC.gz                        Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz                           Pt.GLLBSC.gz
Ag.11.LDA.gz        In.PBE.gz                           Pt.LDA.gz
Ag.11.PBE.gz        In.revPBE.gz                        Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz                          Pt.revPBE.gz
Ag.11.RPBE.gz       I.PBE.gz                            Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz                   Rb.dzp.basis.gz
Ag.GLLBSC.gz        Ir.9.GLLBSC.gz                      Rb.GLLBSC.gz
Ag.LDA.gz           Ir.9.LDA.gz                         Rb.LDA.gz
Ag.PBE.gz           Ir.9.PBE.gz                         Rb.PBE.gz
Ag.revPBE.gz        Ir.9.revPBE.gz                      Rb.revPBE.gz
Ag.RPBE.gz          Ir.9.RPBE.gz                        Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz                     Re.dzp.basis.gz
Al.GLLBSC.gz        I.revPBE.gz                         Re.GLLBSC.gz
Al.LDA.gz           Ir.GLLBSC.gz         

In [ ]:
import os

gpaw_data_path = '/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/'
paw_dataset_file = os.path.join(gpaw_data_path, 'Zr.PBE')

if os.path.exists(paw_dataset_file):
    print(f"The file '{paw_dataset_file}' exists.")
else:
    print(f"The file '{paw_dataset_file}' does not exist.")

The file '/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE' exists.


In [ ]:
!mv /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE.gz

In [ ]:
ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/

Ag.11.dzp.basis.gz  In.GLLBSC.gz                        Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz                           Pt.GLLBSC.gz
Ag.11.LDA.gz        In.PBE.gz                           Pt.LDA.gz
Ag.11.PBE.gz        In.revPBE.gz                        Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz                          Pt.revPBE.gz
Ag.11.RPBE.gz       I.PBE.gz                            Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz                   Rb.dzp.basis.gz
Ag.GLLBSC.gz        Ir.9.GLLBSC.gz                      Rb.GLLBSC.gz
Ag.LDA.gz           Ir.9.LDA.gz                         Rb.LDA.gz
Ag.PBE.gz           Ir.9.PBE.gz                         Rb.PBE.gz
Ag.revPBE.gz        Ir.9.revPBE.gz                      Rb.revPBE.gz
Ag.RPBE.gz          Ir.9.RPBE.gz                        Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz                     Re.dzp.basis.gz
Al.GLLBSC.gz        I.revPBE.gz                         Re.GLLBSC.gz
Al.LDA.gz           Ir.GLLBSC.gz         

In [ ]:
!pip install gpaw ase

!wget https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
!head -n 5 gpaw-setups-24.11.0.tar.gz

--2025-05-06 15:46:18--  https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
Resolving wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)... 130.225.86.27
Connecting to wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)|130.225.86.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71622074 (68M) [application/x-gzip]
Saving to: ‘gpaw-setups-24.11.0.tar.gz.2’

gpaw-setups-24.11.0 100%[===================>]  68.30M  24.7MB/s    in 2.8s    

2025-05-06 15:46:21 (24.7 MB/s) - ‘gpaw-setups-24.11.0.tar.gz.2’ saved [71622074/71622074]

����?-���>��>���?H~/��?��ޯ�߾�~�����?��?��OO��oO��������?���������~�_�~���?�����_�������?���[�ߟ���������+�{�+X����x��}�?�~���L������ػ<����t�����|_^��|������g���G����������xW����:���j�����~�7���������� ^5�?<�G��~W������p���=��~��s��s���5���sK����ۿ���}�x� �����Y������~��	� ��o[}���+������ߏY{����d��������~c�ׇ�z���1՗o��o�q�ا�U�O������������9�o����>���z[�#�=}��w�/W�����:7 ��u|g�u��@g�ag����sw��f�l�
��2���w�����:MJ��[�7�

In [ ]:
!tar -xzf gpaw-setups-24.11.0.tar.gz

**CO Adsorption on Cu {For Fun}**

In [ ]:
!pip install gpaw ase

!wget https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz

!head -n 5 gpaw-setups-24.11.0.tar.gz

!tar -xzf gpaw-setups-24.11.0.tar.gz

import os
os.environ["GPAW_SETUP_PATH"] = "/content/gpaw-setups-24.11.0"

from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW

slab = fcc111('Cu', size=(2, 2, 3), vacuum=10.0)
add_adsorbate(slab, 'C', height=1.85, position='ontop')
add_adsorbate(slab, 'O', height=3.00, position='ontop')
slab.center(axis=2, vacuum=10.0)

fixed_indices = [atom.index for atom in slab if atom.position[2] < slab.positions[:,2].mean()]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts=(2, 2, 1),
    txt='gpaw.log'
)

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)

print("Final energy:", slab.get_potential_energy(), "eV")

--2025-05-06 15:52:21--  https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
Resolving wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)... 130.225.86.27
Connecting to wiki.fysik.dtu.dk (wiki.fysik.dtu.dk)|130.225.86.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71622074 (68M) [application/x-gzip]
Saving to: ‘gpaw-setups-24.11.0.tar.gz.6’

gpaw-setups-24.11.0 100%[===================>]  68.30M  24.3MB/s    in 2.8s    

2025-05-06 15:52:24 (24.3 MB/s) - ‘gpaw-setups-24.11.0.tar.gz.6’ saved [71622074/71622074]

����?-���>��>���?H~/��?��ޯ�߾�~�����?��?��OO��oO��������?���������~�_�~���?�����_�������?���[�ߟ���������+�{�+X����x��}�?�~���L������ػ<����t�����|_^��|������g���G����������xW����:���j�����~�7���������� ^5�?<�G��~W������p���=��~��s��s���5���sK����ۿ���}�x� �����Y������~��	� ��o[}���+������ߏY{����d��������~c�ׇ�z���1՗o��o�q�ا�U�O������������9�o����>���z[�#�=}��w�/W�����:7 ��u|g�u��@g�ag����sw��f�l�
��2���w�����:MJ��[�7�

In [7]:
!pip install --q ase gpaw

In [8]:
!wget -q https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-24.11.0.tar.gz
!tar -xzf gpaw-setups-24.11.0.tar.gz

In [9]:
import os
os.environ["GPAW_SETUP_PATH"] = "/content/gpaw-setups-24.11.0"

In [22]:
import os

setup_path = os.environ["GPAW_SETUP_PATH"]
zr_pbe_path = os.path.join(setup_path, "Zr.PBE")

if not os.path.isfile(zr_pbe_path):
    raise FileNotFoundError(f"'Zr.PBE' not found in {setup_path}. Please verify the file's presence.")

FileNotFoundError: 'Zr.PBE' not found in /content/gpaw-setups-24.11.0. Please verify the file's presence.

#**ZrS2 Adsorption**
#*Au*


In [15]:
from ase.build import add_adsorbate, mx2
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW
from ase.io import write
from google.colab import drive

slab = mx2('ZrS2', kind='1T', a=3.66, vacuum=8.0)

adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Au', height=1.85, position=adsorption_site)

slab.center(axis=2, vacuum=10.0)

z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(mode=PW(400), xc='PBE', kpts=(2, 2, 1), txt='gpaw.log')

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)
# drive.mount('/content/drive')
write('/content/drive/MyDrive/ZrS2(Au).cif',slab)

print("Final potential energy:", slab.get_potential_energy(), "eV")


Final potential energy: -21.795925856491344 eV


In [ ]:
drive.flush_and_unmount

<function google.colab.drive.flush_and_unmount(timeout_ms=86400000)>

In [ ]:
write('/content/drive/MyDrive/ZrS2(Au).cif',slab)

#*Pt*


In [ ]:
from ase.build import add_adsorbate, mx2
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW

slab = mx2('ZrS2', kind='1T', a=3.66, vacuum=8.0)

adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Pt', height=1.85, position=adsorption_site)

slab.center(axis=2, vacuum=10.0)

z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(mode=PW(400), xc='PBE', kpts=(2, 2, 1), txt='gpaw.log')

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)
# drive.mount('/content/drive')
write('/content/drive/MyDrive/ZrS2(Pt).cif',slab)

print("Final potential energy:", slab.get_potential_energy(), "eV")


Final potential energy: -24.723767747956806 eV


#*Bi*

In [ ]:
from ase.build import add_adsorbate, mx2
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW

slab = mx2('ZrS2', kind='1T', a=3.66, vacuum=8.0)

adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Bi', height=1.85, position=adsorption_site)

slab.center(axis=2, vacuum=10.0)

z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(mode=PW(400), xc='PBE', kpts=(2, 2, 1), txt='gpaw.log')

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)
# drive.mount('/content/drive')
write('/content/drive/MyDrive/ZrS2(Bi).cif',slab)

print("Final potential energy:", slab.get_potential_energy(), "eV")


Final potential energy: -23.56010495461476 eV


#**ZrSe2**
#*Au*

In [39]:
from ase.build import add_adsorbate,mx2
from ase.constraints import FixAtoms
from ase.io import write
from ase.optimize import BFGS
from gpaw import GPAW,pw

slab = mx2('ZrSe2',kind='1T',a=3.66,vacuum=8.0)

adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Au', height=1.85, position=adsorption_site)

slab.center(axis=2, vacuum=10.0)

z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(mode=PW(400),xc='PBE',kpts=(2,2,1),txt='gpaw.log')

opt = BFGS(slab,logfile = 'Optimization.log')
opt.run(fmax = 0.05)
write('/content/drive/MyDrive/ZrSe2(Au).cif',slab)
print("Final potential energy:",slab.get_potential_energy(),"eV")

Final potential energy: -20.081233698683523 eV


#*Pt*

In [ ]:
from ase.build import add_adsorbate, mx2
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW
from ase.io import write

slab = mx2('ZrSe2', kind='1T', a=3.66, vacuum=8.0)
adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Pt', height=1.85, position=adsorption_site)

slab.center(axis=2, vacuum=10.0)
z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(mode=PW(400), xc='PBE', kpts=(2, 2, 1), txt='gpaw.log')

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)

write('/content/drive/MyDrive/ZrSe2(Pt).cif', slab)

print("Final potential energy:", slab.get_potential_energy(), "eV")

Final potential energy: -22.88146962177924 eV


#*Bi*

In [ ]:
from ase.build import add_adsorbate, mx2
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from gpaw import GPAW, PW
from ase.io import write

slab = mx2('ZrSe2', kind='1T', a=3.66, vacuum=8.0)
adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Bi', height=1.85, position=adsorption_site)

slab.center(axis=2, vacuum=10.0)
z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices=fixed_indices))

slab.calc = GPAW(mode=PW(400), xc='PBE', kpts=(2, 2, 1), txt='gpaw.log')

opt = BFGS(slab, logfile='optimization.log')
opt.run(fmax=0.05)

write('/content/drive/MyDrive/ZrSe2(Bi).cif', slab)

print("Final potential energy:", slab.get_potential_energy(), "eV")

Final potential energy: -21.889749237286257 eV


#**ZrTe**
#*Au*

In [ ]:
from ase.build import add_adsorbate,mx2
from ase.io import write
from ase.optimize import BFGS
from gpaw import GPAW,PW
from ase.constraints import FixAtoms

slab = mx2('ZrTe2', kind = '1T', a = 3.66, vacuum = 8.0)

adsorption_site = slab[0].position[:2]
add_adsorbate(slab, 'Au', height = 1.85, position = adsorption_site)

slab.center(axis = 2, vacuum = 10.0)
z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices = fixed_indices))

slab.calc = GPAW(mode = PW(400), xc = 'PBE', kpts = (2, 2, 1), txt = 'gpaw.log')

opt = BFGS(slab, logfile = 'optimization.log')
opt.run(fmax = 0.05)

write('/content/drive/MyDrive/ZrTe2(Au).cif', slab)
print("Final Potential Energy : ",slab.get_potential_energy(), "eV")

Final Potential Energy :  -17.25997109414339 eV


#*Pt*

In [ ]:

slab = mx2('ZrTe2', kind = '1T', a = 3.66, vacuum = 8.0)

adsorption_site = slab[0].position[2]
add_adsorbate(slab, 'Pt', height = 1.85, position = adsorption_site)

slab.center(axis = 2, vacuum = 10.0)
z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices = fixed_indices))

slab.calc = GPAW(mode = PW(400), xc = 'PBE', kpts = (2, 2, 1), txt = 'gpaw.log')

opt = BFGS(slab, logfile = 'optimization.log')
opt.run(fmax = 0.05)

write('/content/drive/MyDrive/ZrTe2(Pt).cif', slab)
print("Final Potential Energy : ",slab.get_potential_energy(), "eV")

Final Potential Energy :  -20.45340838224625 eV


#*Bi*

In [ ]:

slab = mx2('ZrTe2', kind = '1T', a = 3.66, vacuum = 8.0)

adsorption_site = slab[0].position[2]
add_adsorbate(slab, 'Bi', height = 1.85, position = adsorption_site)

slab.center(axis = 2, vacuum = 10.0)
z_mean = slab.positions[:, 2].mean()
fixed_indices = [atom.index for atom in slab if atom.position[2] < z_mean]
slab.set_constraint(FixAtoms(indices = fixed_indices))

slab.calc = GPAW(mode = PW(400), xc = 'PBE', kpts = (2, 2, 1), txt = 'gpaw.log')

opt = BFGS(slab, logfile = 'optimization.log')
opt.run(fmax = 0.05)

write('/content/drive/MyDrive/ZrTe2(Bi).cif', slab)
print("Final Potential Energy : ",slab.get_potential_energy(), "eV")

Final Potential Energy :  -19.382612224149394 eV


**ZrS2 (substitution with Au)**

In [49]:
# 1. Installation with proper setup files
# !pip install -q ase gpaw
# !wget -q https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz  # Using older but more reliable version
# !tar -xzf gpaw-setups-0.9.20000.tar.gz

import os
os.environ["GPAW_SETUP_PATH"] = "/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/"

# Verify ALL required setup files exist
print("Verifying setup files:")
!find /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/ -name "Zr.*"
!find /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/ -name "Au.*"
!find /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/ -name "S.*"

# 2. Structure creation and substitution
from ase.build import mx2
from gpaw import GPAW, PW
from ase.optimize import BFGS
from ase.visualize import view

# Create ZrS2 monolayer
slab = mx2(formula='ZrS2', kind='1T', a=3.66, thickness=1, vacuum=10.0)

# Visualize before substitution (optional)
print("\nOriginal structure:")
view(slab)

# Perform substitution
zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
substitution_index = zr_indices[0]  # Selecting first Zr atom
slab[substitution_index].symbol = 'Au'  # Actual substitution

print("\nStructure after Au substitution:")
view(slab)  # Verify substitution visually

# 3. DFT calculation setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
    setups={'Zr': 'pbe', 'S': 'pbe', 'Au': 'pbe'},  # All lowercase
    txt='substitution.log'
)

# 4. Structure relaxation
opt = BFGS(slab, trajectory='optimization.traj')
try:
    opt.run(fmax=0.05)
    print("\nOptimization successful!")
    print("Final Energy:", slab.get_potential_energy(), "eV")

    # Save final structure
    from ase.io import write
    write('final_structure.cif', slab)
    write('final_structure.traj', slab)  # ASE format for future analysis

except Exception as e:
    print("\nError during optimization:", e)
    print("\nTroubleshooting steps:")
    print("1. Verify setup files exist:")
    !ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.*
    print("\n2. Try restarting the runtime (Runtime -> Restart runtime) and run again")

Verifying setup files:
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.revPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.dzp.basis.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.RPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.LDA.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.GLLBSC.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Zr.PBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.LDA.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.GLLBSC.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.RPBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.PBE.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.dzp.basis.gz
/usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/Au.revPBE.gz
/usr/local/lib/python3/dist

In [11]:
!ls /usr/local/lib/python3/dist-packages/gpaw/data/gpaw-setups-24.11.0/
# !ls /content/gpaw-setups-0.9.20000/Zr.pbe

Ag.11.dzp.basis.gz  In.GLLBSC.gz			Pt.dzp.basis.gz
Ag.11.GLLBSC.gz     In.LDA.gz				Pt.GLLBSC.gz
Ag.11.LDA.gz	    In.PBE.gz				Pt.LDA.gz
Ag.11.PBE.gz	    In.revPBE.gz			Pt.PBE.gz
Ag.11.revPBE.gz     In.RPBE.gz				Pt.revPBE.gz
Ag.11.RPBE.gz	    I.PBE.gz				Pt.RPBE.gz
Ag.dzp.basis.gz     Ir.9.dzp.basis.gz			Rb.dzp.basis.gz
Ag.GLLBSC.gz	    Ir.9.GLLBSC.gz			Rb.GLLBSC.gz
Ag.LDA.gz	    Ir.9.LDA.gz				Rb.LDA.gz
Ag.PBE.gz	    Ir.9.PBE.gz				Rb.PBE.gz
Ag.revPBE.gz	    Ir.9.revPBE.gz			Rb.revPBE.gz
Ag.RPBE.gz	    Ir.9.RPBE.gz			Rb.RPBE.gz
Al.dzp.basis.gz     Ir.dzp.basis.gz			Re.dzp.basis.gz
Al.GLLBSC.gz	    I.revPBE.gz				Re.GLLBSC.gz
Al.LDA.gz	    Ir.GLLBSC.gz			Re.LDA.gz
Al.PBE.gz	    Ir.LDA.gz				Re.PBE.gz
Al.revPBE.gz	    Ir.PBE.gz				Re.revPBE.gz
Al.RPBE.gz	    I.RPBE.gz				Re.RPBE.gz
Ar.dzp.basis.gz     Ir.revPBE.gz			Rh.9.dzp.basis.gz
Ar.GLLBSC.gz	    Ir.RPBE.gz				Rh.9.GLLBSC.gz
Ar.LDA.gz	    K.dzp.basis.gz			Rh.9.LDA.gz
Ar.PBE.gz	    K.GLLBSC.gz				Rh.9.PBE.gz
Ar.revPBE.gz	    K.LDA.g

In [15]:
from ase.build import mx2
from gpaw import GPAW, PW
from ase.optimize import BFGS

slab = mx2('ZrS2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Au'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrS2Au.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:24:48       -9.845269        1.800877
BFGS:    1 11:25:12       -9.934916        1.679547
BFGS:    2 11:25:41      -10.360583        0.351077
BFGS:    3 11:26:08      -10.373023        0.110939
BFGS:    4 11:26:35      -10.374459        0.000572
Optimization successful!
Final Energy: -10.326958187170378 eV


In [16]:
slab = mx2('ZrS2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Pt'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrS2Pt.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:27:33      -14.033379        3.247256
BFGS:    1 11:28:02      -14.325045        3.043052
BFGS:    2 11:28:39      -15.286863        1.648272
BFGS:    3 11:29:24      -15.544855        0.467804
BFGS:    4 11:30:07      -15.562868        0.054914
BFGS:    5 11:30:28      -15.563122        0.001927
Optimization successful!
Final Energy: -15.562940100524612 eV


In [17]:
slab = mx2('ZrS2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Bi'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrS2Bi.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:31:14      -12.505194        0.830166
BFGS:    1 11:31:31      -12.523387        0.697148
BFGS:    2 11:31:51      -12.575835        0.141650
BFGS:    3 11:32:11      -12.577924        0.016751
Optimization successful!
Final Energy: -12.525256681019112 eV


**ZrSe2 substitution of Au,Pt,Bi**

In [18]:
slab = mx2('ZrSe2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Au'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrSe2Au.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:32:52       -9.552278        0.549448
BFGS:    1 11:33:07       -9.560523        0.501979
BFGS:    2 11:33:30       -9.598032        0.090126
BFGS:    3 11:33:46       -9.598941        0.022163
Optimization successful!
Final Energy: -9.549574091411417 eV


In [19]:
slab = mx2('ZrSe2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Pt'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrSe2Pt.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:34:43      -13.867821        2.142036
BFGS:    1 11:35:07      -13.993024        1.945040
BFGS:    2 11:35:47      -14.426662        0.054575
BFGS:    3 11:36:04      -14.426873        0.014135
Optimization successful!
Final Energy: -14.426021994727709 eV


In [20]:
slab = mx2('ZrSe2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Bi'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrSe2Bi.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:36:46      -10.517428        2.738008
BFGS:    1 11:37:05      -10.713663        2.284021
BFGS:    2 11:37:35      -11.225102        0.541278
BFGS:    3 11:37:56      -11.271142        0.219971
BFGS:    4 11:38:16      -11.282247        0.052866
BFGS:    5 11:38:30      -11.283098        0.002635
Optimization successful!
Final Energy: -11.209628041156678 eV


**ZrTe2 substitution using Au,Pt,Bi**

In [21]:
slab = mx2('ZrTe2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Au'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrTe2Au.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:39:11       -8.691121        2.056644
BFGS:    1 11:39:29       -8.801738        1.727797
BFGS:    2 11:40:06       -9.138164        0.582813
BFGS:    3 11:40:26       -9.201004        0.230626
BFGS:    4 11:40:45       -9.214636        0.029478
Optimization successful!
Final Energy: -9.173993427456164 eV


In [22]:
slab = mx2('ZrTe2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Pt'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrTe2Pt.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:41:45      -13.351688        0.479056
BFGS:    1 11:42:04      -13.357848        0.426612
BFGS:    2 11:42:34      -13.381939        0.020539
Optimization successful!
Final Energy: -13.331951676056422 eV


In [23]:
slab = mx2('ZrTe2', kind='1T', a=3.66, vacuum=10.0)

zr_indices = [i for i, atom in enumerate(slab) if atom.symbol == 'Zr']
slab[zr_indices[0]].symbol = 'Bi'

# DFT setup
slab.calc = GPAW(
    mode=PW(400),
    xc='PBE',
    kpts={'size': (4, 4, 1), 'gamma': True},
      # Use this instead of element-specific setups
    txt='substitution.log'
)

opt = BFGS(slab, trajectory='optimization_ZrTe2Bi.traj')
opt.run(fmax=0.05)

print("Optimization successful!")
print("Final Energy:", slab.get_potential_energy(), "eV")

# Save results
from ase.io import write
write('final_structure.cif', slab)
write('final_structure.traj', slab)

      Step     Time          Energy          fmax
BFGS:    0 11:43:18       -6.152641        6.655309
BFGS:    1 11:43:45       -7.300760        5.491478
BFGS:    2 11:44:16       -9.054280        3.266616
BFGS:    3 11:44:44      -10.012870        1.525241
BFGS:    4 11:45:32      -10.282314        0.146422
BFGS:    5 11:45:49      -10.286294        0.068749
BFGS:    6 11:46:04      -10.287558        0.006863
Optimization successful!
Final Energy: -10.252072930316716 eV


In [26]:
from ase.io import read,write
atoms = read('/content/optimization_ZrS2Au.traj')
write('/content/drive/MyDrive/Op_ZrS2_Au.cif',atoms)
atoms = read('/content/optimization_ZrS2Pt.traj')
write('/content/drive/MyDrive/Op_ZrS2_Pt.cif',atoms)
atoms = read('/content/optimization_ZrS2Bi.traj')
write('/content/drive/MyDrive/Op_ZrS2_Bi.cif',atoms)

In [27]:
atoms = read('/content/optimization_ZrSe2Au.traj')
write('/content/drive/MyDrive/Op_ZrSe2_Au.cif',atoms)
atoms = read('/content/optimization_ZrSe2Pt.traj')
write('/content/drive/MyDrive/Op_ZrSe2_Pt.cif',atoms)
atoms = read('/content/optimization_ZrSe2Bi.traj')
write('/content/drive/MyDrive/Op_ZrSe2_Bi.cif',atoms)

In [28]:
atoms = read('/content/optimization_ZrTe2Au.traj')
write('/content/drive/MyDrive/Op_ZrTe2_Au.cif',atoms)
atoms = read('/content/optimization_ZrTe2Pt.traj')
write('/content/drive/MyDrive/Op_ZrTe2_Pt.cif',atoms)
atoms = read('/content/optimization_ZrTe2Bi.traj')
write('/content/drive/MyDrive/Op_ZrTe2_Bi.cif',atoms)